# Código TP2: Selección de Variables

In [1]:
from typing import Any, Callable, Dict, List
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.svm import LinearSVC
from sklearn.utils import resample

## Función general de uso de distintos métodos para forward ranking

In [2]:
#-------------------------------------------------------------------------------
# AVISO: codigo de demostracion
# No es optimo, no es la mejor solucion
#-------------------------------------------------------------------------------


def forward_ranking(
        X: np.ndarray, y: np.ndarray,
        method: Callable[[np.ndarray, np.ndarray, Dict[str, Any]], float],
        verbosity: int = 0, **kwargs) -> List[int]:
    """
    Método Wrapper Forward Selection/Ranking.
    Comienza con una lista vacía e iterativamente añade la variable que junto
    a las ya seleccionadas retorna el menor error según el método provisto
    como argumento.

    Argumentos:
        X: lista de N inputs de dimensión M como array numpy de shape (N, M).
        y: lista de N targets como array numpy de shape (N,).
        method: función que estima el classification error para un clasificador
            dado con los datos dados; debe poder tomar una lista de inputs y
            una lista de targets y los argumentos pasados por "**kwargs", y
            devolver el error del modelo en el intervalo ( 0 , 1 ).
        verbosity: nivel de verbosidad de 0 a 3 que controla cuanta información
            se imprime por pantalla; a mayor número, más información imprime.
        **kwargs: parámetros adicionales a pasarle a la función de estimación.

    Retorna:
        orden de importancia de las variables (descendiente, numeradas desde cero)
    """
    _, max_feat = X.shape  # total de features
    num_feat = 0           # numero actual de features
    # lista para guardar los features elegidos, inicializo como llegaron
    list_feat = np.arange(0, max_feat)

    # ranking inicial: elijo la variable con menor error de predicción
    class_error = np.zeros(max_feat)
    # para cada indice i creo el dataset con la variable sola, entreno el
    # modelo, y le mido el error, que lo guardo en class_error[i]
    for i in range(max_feat):
        X_train = X[:, [i]]  # preservamos estructura de matriz
        class_error[i] = method(X_train, y, **kwargs)

    if verbosity > 2:
        print(f'\nIndividual Classes Error:\n{class_error}')

    # guardo la variable con mínimo error como primera en mi lista de elegidas
    # guardo una lista keep_feat con las que me quedan para seguir eligiendo
    list_feat[0] = np.argmin(class_error)
    keep_feat = np.argsort(class_error)[1:]
    num_feat += 1

    if verbosity > 1:
        print(f'\nFirst feature: {list_feat[0]}')

    # loop principal
    # a cada paso agrego todas las variables disponibles, de a una, les mido el
    # error y me quedo con la de mínimo error hasta llegar a meter todas
    while num_feat < max_feat:
        class_error = np.zeros(max_feat - num_feat)

        # class_error guarda el error de cada modelo restante
        for i in range(max_feat - num_feat):
            features = np.concatenate((list_feat[0:num_feat], [keep_feat[i]]))
            X_train = X[:, features]
            class_error[i] = method(X_train, y, **kwargs)

        if verbosity > 2:
            print(f'\nFeatures:\n{keep_feat}\nErrors:\n{class_error}')

        # me quedo con el modelo de minimo error, guardo ese feature en la lista
        # de las elegidas, lo saco de la lista de las que me quedan
        best_index = np.argmin(class_error)
        list_feat[num_feat] = keep_feat[best_index]

        if verbosity > 1:
            print(f'\n------------\nStep {num_feat+1}\nFeature {best_index}')

        keep_feat = np.delete(keep_feat, best_index)

        if verbosity > 2:
            print(f'\nNew search list {keep_feat}')

        num_feat += 1

    if verbosity > 1:
        print(f'\n------------\nFinal ranking of features: {list_feat}\n')

    return list_feat


## Métodos para entrenamiento y estimación de error

In [4]:
def error_rate(A, B):
    """  Función de ayuda para calcular error """
    return sum(A != B) / len(B)


def RF_error_estimation(
        X, y, equalize_classes = True, total_trees = 500, mtry = 0):
    """
    Random forest error estimation (Out-Of-Bag) for greedy search
    """
    if mtry < 1:
        mtry = int(np.floor(np.sqrt(X.shape[1])))
    class_weights = None
    if equalize_classes:
        class_weights = 'balanced'
    random_forest = RandomForestClassifier(
        n_estimators=total_trees, max_features=mtry,
        bootstrap=True, class_weight=class_weights, oob_score=True)
    random_forest.fit(X, y)
    return 1 - random_forest.oob_score_  # final out-of-bag error


def LDA_error_estimation(X, y):
    """
    LDA error estimation (Leave-One-Out) for greedy search
    """
    loo = LeaveOneOut()
    y_pred = np.empty_like(y)  # empty results array
    for train_index, test_index in loo.split(X):
        lda = LinearDiscriminantAnalysis()
        lda.fit(X[train_index], y[train_index])
        y_pred[test_index] = lda.predict(X[test_index])
    return error_rate(y, y_pred)


def SVM_error_estimation(
        X, y, C = 1.0, cross = 4):
    """
    SVM error estimation (internal Cross-Validation) for greedy search
    """
    svm = LinearSVC(C=C)
    return 1 - cross_val_score(svm, X, y, cv=cross).mean()


## Métodos de Ranking

In [3]:
def RF_ranking_method(
        X, y, equalize_classes = True, total_trees = 500, mtry = 0):
    """
    Random forest ranking method for rfe
    """
    if mtry < 1:
        mtry = int(np.floor(np.sqrt(X.shape[1])))

    class_weights = None
    if equalize_classes:
        class_weights = 'balanced'

    random_forest = RandomForestClassifier(
        n_estimators=total_trees, max_features=mtry,
        bootstrap=True, class_weight=class_weights, oob_score=True)
    random_forest.fit(X, y)

    importance_values = random_forest.feature_importances_
    feats = np.argsort(importance_values)
    imp = importance_values[feats]

    return (feats, imp)


def LSVM_ranking_method(X, y, C = 100):
    """
    Linear svm ranking method for rfe. Multiclass
    """
    svm = LinearSVC(C=C)
    svm.fit(X, y)

    weights = np.zeros(len(np.unique(y)))
    # coef_ is of shape (1, nfeat) if 2-class, (nclass, nfeat) otherwise
    coefs = np.abs(svm.coef_)
    # Sumar la importancia de cada característica a través de los modelos
    w = np.sum(coefs, axis=0)

    feats = np.argsort(w)
    imp = w[feats]
    return (feats, imp)


## Ejemplos de Uso

In [5]:
# Includes de distintos métodos
from sklearn import datasets


# Exploramos dataset iris
iris = datasets.load_iris()
print(iris.DESCR)

print("Iris Inputs (X) shape:", iris.data.shape)
print("Iris Targets (y) shape:", iris.target.shape)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [26]:
# Probamos el metodo forward-ranking con IRIS
iris_forward_rf = forward_ranking(
    iris.data, iris.target,
    method=RF_error_estimation, total_trees=100,
    equalize_classes=False, verbosity=0)
print(f'RF: {iris_forward_rf}')

iris_forward_rf2 = forward_ranking(
    iris.data, iris.target,
    method=RF_error_estimation, total_trees=100,
    equalize_classes=True, verbosity=0)
print(f'RF2: {iris_forward_rf2}')

iris_forward_lda = forward_ranking(
    iris.data, iris.target, method=LDA_error_estimation)
print(f'LDA: {iris_forward_lda}')

iris_forward_svm = forward_ranking(
    iris.data, iris.target, method=SVM_error_estimation)
print(f'SVM: {iris_forward_svm}')

RF: [3 2 0 1]
RF2: [3 2 1 0]
LDA: [3 2 0 1]
SVM: [3 1 2 0]


In [9]:
# Creador de ruido uniforme multimodal
def crea_ruido_unif(n = 100, d = 2):
    X = np.random.uniform(-1, 1, size=(2*n,d))
    y = np.repeat([1,-1], [n,n])
    return (X,y)  # NOTA: difiere de implem. R


# Crea dataset datosA
# dataset artificial con orden de importancia: 7-5-3-1
d = 10
n = 1000
datosAX, datosAy = crea_ruido_unif(n=n, d=d)
# tomar 50% de los datos al azar, y hacer que la clase sea el signo
# de la 8 variable
shuffle = np.random.permutation(len(datosAy))
sub = shuffle[:int(len(datosAy) * 0.5)]
datosAy[sub] = np.sign(datosAX[sub, 7])
# tomar 20% de los datos al azar (fuera de los anteriores), y hacer
# que la clase sea el signo de la 6 variable
sub = shuffle[int(len(datosAy) * 0.5):int(len(datosAy) * 0.7)]
datosAy[sub] = np.sign(datosAX[sub, 5])
# tomar 10% de los datos al azar, y hacer que la clase sea el signo de
# la 4 variable
sub = shuffle[int(len(datosAy) * 0.7):int(len(datosAy) * 0.8)]
datosAy[sub] = np.sign(datosAX[sub, 3])
# tomar 5% de los datos al azar, y hacer que la clase sea el signo de
# la 2 variable
sub = shuffle[int(len(datosAy) * 0.8):int(len(datosAy) * 0.85)]
datosAy[sub] = np.sign(datosAX[sub, 1])


# Crea dataset datosB
# dataset artificial con dos variables relevantes (0-1) y dos variables
# que son importantes pero que no resuelven el problema (2-3)
d = 8
n = 1000
datosBX, datosBy = crea_ruido_unif(n=n, d=d)
# hacer que la clase sea el xor de las 2 primeras variables (es usando el signo)
datosBy = np.sign(datosBX[:, 0] * datosBX[:, 1])
# hacer que las variables 3 y 4 tengan un 50% de correlacion con la clase
shuffle = np.random.permutation(len(datosBy))
sub = shuffle[:int(len(datosBy) * 0.5)]
datosBX[sub, 2] = np.abs(datosBX[sub, 2] * datosBy[sub])
shuffle = np.random.permutation(len(datosBy))
sub = shuffle[:int(len(datosBy) * 0.5)]
datosBX[sub, 3] = np.abs(datosBX[sub, 3] * datosBy[sub])


# Ejercicio 1

## Wrapper greedy backward 

In [24]:
def backward_ranking(
        X: np.ndarray, y: np.ndarray,
        method: Callable[[np.ndarray, np.ndarray, Dict[str, Any]], float],
        verbosity: int = 0, **kwargs) -> List[int]:
    _, max_feat = X.shape  # total de features
    num_feat = 0           # numero actual de features
    keep_feat = np.arange(0, max_feat)  # lista de los que me quedan para seguir eligiendo
    del_feat = np.arange(0, max_feat)  # lista para guardar los features eliminados

    # loop principal
    # a cada paso agrego todas las variables disponibles, de a una, les mido el
    # error y me quedo con la de mínimo error hasta llegar a meter todas
    while num_feat < max_feat-1:
        class_error = np.zeros(max_feat - num_feat)

        # class_error guarda el error de cada modelo restante
        for i in range(max_feat - num_feat):
            features = np.delete(keep_feat, i)
            X_train = X[:, features]
            class_error[i] = method(X_train, y, **kwargs)

        if verbosity > 2:
            print(f'\nFeatures:\n{keep_feat}\nErrors:\n{class_error}')

        # saco la variable del modelo con menos error (porque es la menos importante)
        worst_index = np.argmin(class_error)
        del_feat[num_feat] = keep_feat[worst_index]

        if verbosity > 1:
            print(f'\n------------\nStep {num_feat+1}\nFeature {worst_index}')

        keep_feat = np.delete(keep_feat, worst_index)

        if verbosity > 2:
            print(f'\nNew search list {keep_feat}')

        num_feat += 1

    del_feat[num_feat] = keep_feat[0]
    
    if verbosity > 1:
        print(f'\n------------\nFinal ranking of features: {np.flip(del_feat)}\n')

    return np.flip(del_feat)


In [43]:
# Probamos el metodo backward-ranking con IRIS
iris_backward_rf = backward_ranking(
    iris.data, iris.target,
    method=RF_error_estimation, total_trees=100,
    equalize_classes=False, verbosity=0)
print(f'RF: {iris_backward_rf}')

iris_backward_rf2 = backward_ranking(
    iris.data, iris.target,
    method=RF_error_estimation, total_trees=100,
    equalize_classes=True, verbosity=0)
print(f'RF2: {iris_backward_rf2}')

iris_backward_lda = backward_ranking(
    iris.data, iris.target, method=LDA_error_estimation)
print(f'LDA: {iris_backward_lda}')

iris_backward_svm = backward_ranking(
    iris.data, iris.target, method=SVM_error_estimation)
print(f'SVM: {iris_backward_svm}')

RF: [3 2 0 1]
RF2: [3 2 1 0]
LDA: [3 2 0 1]
SVM: [2 1 0 3]


## Filter con test no-paramétrico (Kruskal-Wallis) 

In [44]:
from scipy.stats import kruskal

def filter_kruskal(
        X: np.ndarray, y: np.ndarray,
        verbosity: int = 0, **kwargs) -> List[int]:
    _, max_feat = X.shape  # total de features

    stats = np.arange(0, max_feat)
    #list_feat = np.arange(0, max_feat)
    for num_feat in range(max_feat):
        x = X[:, num_feat]
        stats[num_feat] = kruskal(x,y).statistic

    if verbosity > 2:
        print(f'\n------------\nStats of features: {stats}\n')
    order = np.argsort(stats)
    #list_feat = list_feat[order]
    if verbosity > 1:
        print(f'\n------------\nFinal ranking of features: {order}\n')
        
    return order

In [45]:
iris_kruskal = filter_kruskal(
    iris.data, iris.target,
    verbosity=0)
print(f'Kruskal: {iris_kruskal}')

Kruskal: [3 2 1 0]


## Eliminación recursiva de características

In [ ]:
def rfe_ranking(
        X: np.ndarray, y: np.ndarray,
        method: Callable[[np.ndarray, np.ndarray, Dict[str, Any]], float],
        verbosity: int = 0, **kwargs) -> List[int]:
    _, max_feat = X.shape  # total de features
    num_feat = 0           # numero actual de features
    keep_feat = np.arange(0, max_feat)  # lista de los que me quedan para seguir eligiendo
    del_feat = np.arange(0, max_feat)  # lista para guardar los features eliminados

    # loop principal
    # a cada paso agrego todas las variables disponibles, de a una, les mido el
    # error y me quedo con la de mínimo error hasta llegar a meter todas
    while num_feat < max_feat-1:
        class_error = np.zeros(max_feat - num_feat)

        # class_error guarda el error de cada modelo restante
        for i in range(max_feat - num_feat):
            features = np.delete(keep_feat, i)
            X_train = X[:, features]
            class_error[i] = method(X_train, y, **kwargs)

        if verbosity > 2:
            print(f'\nFeatures:\n{keep_feat}\nErrors:\n{class_error}')

        # saco la variable del modelo con menos error (porque es la menos importante)
        worst_index = np.argmin(class_error)
        del_feat[num_feat] = keep_feat[worst_index]

        if verbosity > 1:
            print(f'\n------------\nStep {num_feat+1}\nFeature {worst_index}')

        keep_feat = np.delete(keep_feat, worst_index)

        if verbosity > 2:
            print(f'\nNew search list {keep_feat}')

        num_feat += 1

    del_feat[num_feat] = keep_feat[0]
    
    if verbosity > 1:
        print(f'\n------------\nFinal ranking of features: {np.flip(del_feat)}\n')

    return np.flip(del_feat)